# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [13]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
city_weather_df = pd.read_csv("output_data/cities.csv")
# city_weather.head()
city_weather_data_df = city_weather_df.drop(columns = 'Unnamed: 0')
city_weather_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Vila Velha,-20.3297,-40.2925,64.35,94,40,3.44,BR,1624169083
1,Mataura,-46.1927,168.8643,45.48,95,76,3.51,NZ,1624168909
2,Cap Malheureux,-19.9842,57.6142,75.47,59,38,11.99,MU,1624169083
3,Wu’an,31.6836,112.0044,73.04,62,100,2.64,CN,1624169084
4,Rikitea,-23.1203,-134.9692,75.20,77,10,19.93,PF,1624169084
...,...,...,...,...,...,...,...,...,...
574,Taquara,-29.6506,-50.7806,58.78,89,100,1.39,BR,1624169243
575,Kaeo,-35.1000,173.7833,58.32,93,33,15.59,NZ,1624169243
576,Evensk,61.9500,159.2333,53.17,63,78,2.64,RU,1624169243
577,Yebaishou,41.3975,119.6408,81.25,26,59,15.48,CN,1624169244


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Lat and Lng as locations
locations = city_weather_data_df[["Lat", "Lng"]]

# Add humidity as the weight

humid_weight = city_weather_data_df["Humidity"]


In [25]:
# Create a humidity Heatmap layer

fig = gmaps.figure(center=(0.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humid_weight, dissipating=False, max_intensity=100,
                                point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [29]:
# Narrow down the cities to fit weather conditions: max temp between 70 and 80, wind speed less than 10 mph,
# and cloudiness equals to 0. 
selected_cities_df = city_weather_data_df.loc[(city_weather_data_df["Wind Speed"] <= 10) & 
                                            (city_weather_data_df["Cloudiness"] == 0) &
                                            (city_weather_data_df["Max Temp"] >= 70) & 
                                            (city_weather_data_df["Max Temp"] <= 80)].dropna()

selected_cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
48,Nikolskoye,59.7035,30.7861,78.80,49,0,7.76,RU,1624169095
55,Tūkrah,32.5341,20.5791,75.02,71,0,3.47,LY,1624169097
56,Talas,38.6908,35.5538,72.27,46,0,8.05,TR,1624169097
78,Racibórz,50.0919,18.2193,77.22,78,0,4.14,PL,1624169105
98,Podgorica,42.4411,19.2636,77.50,73,0,1.14,ME,1624169111
101,Novouzensk,50.4592,48.1431,78.22,43,0,5.95,RU,1624169113
129,Methóni,36.8197,21.7049,77.22,66,0,9.51,GR,1624169120
199,Belyy Yar,53.6039,91.3903,78.69,50,0,2.24,RU,1624169138
222,Marsá Maţrūḩ,31.3525,27.2453,74.71,68,0,8.03,EG,1624169143
240,Koróni,36.8000,21.9500,78.08,63,0,2.57,GR,1624169148


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [31]:
# Create a varibale named hotel_df
hotel_df = selected_cities_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add an empty column named "Hotel Name" to the DataFrame
hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name
48,Nikolskoye,RU,59.7035,30.7861,
55,Tūkrah,LY,32.5341,20.5791,
56,Talas,TR,38.6908,35.5538,
78,Racibórz,PL,50.0919,18.2193,
98,Podgorica,ME,42.4411,19.2636,
101,Novouzensk,RU,50.4592,48.1431,
129,Methóni,GR,36.8197,21.7049,
199,Belyy Yar,RU,53.6039,91.3903,
222,Marsá Maţrūḩ,EG,31.3525,27.2453,
240,Koróni,GR,36.8000,21.9500,


In [33]:
# Set up search parameters dictionary

url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "lodging",
          "keyword" : "lodging",
          "radius" : 5000,
          "key" : g_key}

In [44]:
# Hit the Google Places API for each city's coordinates

for index, row in hotel_df.iterrows():
    
 # set up to retrieve city name, lat, lnt from the dataframe
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(url, params=params).json()
    
    results = response['results']
    
    try:
        print(f"Closest hotel to {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        
    except IndexError:
        print(f"No hotel with 5km to {city_name}... skipping.")


Retrieving Results for Index 48: Nikolskoye.
No hotel with 5km to Nikolskoye... skipping.
Retrieving Results for Index 55: Tūkrah.
Closest hotel to Tūkrah is مصيف مرحبا للعائلات.
Retrieving Results for Index 56: Talas.
Closest hotel to Talas is PRIME INN MAXX.
Retrieving Results for Index 78: Racibórz.
Closest hotel to Racibórz is Willa Starowiejska.
Retrieving Results for Index 98: Podgorica.
Closest hotel to Podgorica is Hotel Podgorica.
Retrieving Results for Index 101: Novouzensk.
No hotel with 5km to Novouzensk... skipping.
Retrieving Results for Index 129: Methóni.
Closest hotel to Methóni is Afentiko Pigadi - Luxury Villas.
Retrieving Results for Index 199: Belyy Yar.
No hotel with 5km to Belyy Yar... skipping.
Retrieving Results for Index 222: Marsá Maţrūḩ.
Closest hotel to Marsá Maţrūḩ is Negresco Hotel.
Retrieving Results for Index 240: Koróni.
Closest hotel to Koróni is Colonides Beach Hotel Ξενοδοχείο.
Retrieving Results for Index 256: Waddān.
No hotel with 5km to Waddān...

In [62]:
# Store Hotel results in the DataFrame

hotel_df

,City,Country,Lat,Lng,Hotel Name
48,Nikolskoye,RU,59.7035,30.7861,Tourist House - Sablino
55,Tūkrah,LY,32.5341,20.5791,مصيف مرحبا للعائلات
56,Talas,TR,38.6908,35.5538,PRIME INN MAXX
78,Racibórz,PL,50.0919,18.2193,Willa Starowiejska
98,Podgorica,ME,42.4411,19.2636,Hotel Podgorica
101,Novouzensk,RU,50.4592,48.1431,
129,Methóni,GR,36.8197,21.7049,Afentiko Pigadi - Luxury Villas
199,Belyy Yar,RU,53.6039,91.3903,
222,Marsá Maţrūḩ,EG,31.3525,27.2453,Negresco Hotel
240,Koróni,GR,36.8000,21.9500,Colonides Beach Hotel Ξενοδοχείο


In [66]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [68]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))